In [5]:
def load_floats_from_file(filename):
    floats = []
    with open(filename, 'r') as file:
        for line in file:
            try:
                value = float(line.strip())
                floats.append(value)
            except ValueError:
                print(f"Skipping invalid line: {line}")
    return floats


In [9]:
coherent_array = load_floats_from_file("coherent.txt")
stochastic_array = load_floats_from_file("stochastic.txt")

In [14]:
array1 = coherent_array
array2 = stochastic_array

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Define the window size
window_size = 10

# Create input windows for each array
windows_array1 = [array1[i:i+window_size] for i in range(len(array1) - window_size + 1)]
windows_array2 = [array2[i:i+window_size] for i in range(len(array2) - window_size + 1)]

# Combine the windows and labels
windows = np.concatenate([windows_array1, windows_array2])
labels = np.concatenate([np.zeros(len(windows_array1)), np.ones(len(windows_array2))])

# Split the data into train, validation, and test sets
windows_train, windows_test, labels_train, labels_test = train_test_split(windows, labels, test_size=0.2, random_state=42)
windows_train, windows_val, labels_train, labels_val = train_test_split(windows_train, labels_train, test_size=0.2, random_state=42)

# Convert the windows and labels to TensorFlow tensors
windows_train = tf.convert_to_tensor(windows_train, dtype=tf.float32)
labels_train = tf.convert_to_tensor(labels_train, dtype=tf.float32)
windows_val = tf.convert_to_tensor(windows_val, dtype=tf.float32)
labels_val = tf.convert_to_tensor(labels_val, dtype=tf.float32)
windows_test = tf.convert_to_tensor(windows_test, dtype=tf.float32)
labels_test = tf.convert_to_tensor(labels_test, dtype=tf.float32)

# Create a sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((window_size, 1), input_shape=(window_size,)),
    tf.keras.layers.Conv1D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with validation data
model.fit(windows_train, labels_train, epochs=200, batch_size=32, validation_data=(windows_val, labels_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(windows_test, labels_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [19]:
# Make predictions on the test data
predictions = model.predict(windows_test)

# Convert the predictions to binary labels
predicted_labels = np.round(predictions).flatten()

# Compare predicted labels with true labels
accuracy = np.mean(predicted_labels == labels_test.numpy())

# Print the accuracy
print("Test Accuracy:", accuracy)


63/63 [==============================] - 0s 640us/step
Test Accuracy: 0.6539809714571858
